In [ ]:
!pip install tensorflow==2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 42.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing 

In [ ]:
!pip install --upgrade deepctr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.2/141.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.8.0 which is incompatible.


In [ ]:
import pandas as pd
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score

# 와인 데이터셋 로드
file_path = '/content/drive/MyDrive/wine_preprocessed_test.csv'
df = pd.read_csv(file_path, encoding='cp949')

# 'food' 열 삭제
df = df.drop(columns=['food'])

# 'type_id' 열 값 변경
df.loc[df['type_id'] == 7, 'type_id'] = 5
df.loc[df['type_id'] == 24, 'type_id'] = 6

df['grapes'] = df['grapes'].apply(lambda x: eval(x)[0] if isinstance(x, str) else x)

# 사용자 설문조사 응답 받기
print("당신의 와인 취향을 선택해주세요. (1~6)")
print("와인의 종류를 선택해주세요.")
print("1: 레드 와인")
print("2: 화이트 와인")
print("3: 스파클링 와인")
print("4: 로제 와인")
print("5: 디저트 와인")
print("6: 포트 와인")
wine_type = int(input())

# 사용자 설문조사 응답 받기
print("와인의 특성을 선택해주세요.")
print("1. Body (1-5)")
body = float(input())
print("2. Alcohol Content (%)")
alcohol = float(input())
print("3. Acidity Level (1-5)")
acidity = float(input())
print("4. Sweetness (1-5)")
sweetness = float(input())
print("5. Tannin (1-5)")
tannin = float(input())

# 설문조사 응답을 기반으로 필터링
filtered_wines = df[
    (df['type_id'] == wine_type) &
    (df['body'] >= body - 1) &
    (df['body'] <= body + 1) &
    (df['alcohol'] >= alcohol - 1) &
    (df['alcohol'] <= alcohol + 1) &
    (df['acidity_y'] >= acidity - 1) &
    (df['acidity_y'] <= acidity + 1) &
    (df['sweetness'] >= sweetness - 1) &
    (df['sweetness'] <= sweetness + 1) &
    (df['tannin'] >= tannin - 1) &
    (df['tannin'] <= tannin + 1)
]

if len(filtered_wines) == 0:
    print("선택한 조건에 해당하는 와인이 없습니다.")
else:
    # DeepFM 모델에 사용할 feature 설정
    sparse_features = ['type_id']
    dense_features = ['body', 'alcohol', 'acidity_y', 'sweetness', 'tannin']
    target = ['like']

    # feature의 유형에 따라 SparseFeat와 DenseFeat 객체 생성
    sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df[feat].nunique() + 2, embedding_dim=4) for feat in sparse_features]
    dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

    # 모델 입력 특성과 모델 입력 특성의 이름을 얻음
    feature_names = get_feature_names(sparse_feature_columns + dense_feature_columns)

    # 데이터셋을 훈련셋과 테스트셋으로 분할
    train, test = train_test_split(filtered_wines, test_size=0.2, random_state=42)

    # 모델 학습에 사용할 데이터셋 생성
    train_model_input = {name: train[name] for name in feature_names}
    test_model_input = {name: test[name] for name in feature_names}

    # DeepFM 모델 생성
    model = DeepFM(sparse_feature_columns, dense_feature_columns, task='binary', l2_reg_linear=0.001, l2_reg_embedding=0.001, l2_reg_dnn=0.001)

    # 모델 컴파일
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # 모델 훈련
    model.fit(train_model_input, train[target].values, batch_size=64, epochs=20, validation_split=0.2)

    # 테스트셋에 대한 예측
    pred_ans = model.predict(test_model_input, batch_size=64)

    # AUC 계산
    test_auc = roc_auc_score(test[target].values, pred_ans)
    print("Test AUC: {:.4f}".format(test_auc))

    # 예측 결과 출력
    test['predicted_like'] = pred_ans
    top_10_wines = test.sort_values(by='predicted_like', ascending=False).head(10)
    print("조건에 맞는 상위 10개의 와인:")
    print(top_10_wines[['name', 'body', 'alcohol', 'acidity_y', 'sweetness', 'tannin']])

당신의 와인 취향을 선택해주세요. (1~6)
와인의 종류를 선택해주세요.
1: 레드 와인
2: 화이트 와인
3: 스파클링 와인
4: 로제 와인
5: 디저트 와인
6: 포트 와인
1
와인의 특성을 선택해주세요.
1. Body (1-5)
3
2. Alcohol Content (%)
12
3. Acidity Level (1-5)
3
4. Sweetness (1-5)
3
5. Tannin (1-5)
3
Epoch 1/20
4/4 [==============================] - 1s 122ms/step - loss: 0.7658 - accuracy: 0.8131 - val_loss: 0.7639 - val_accuracy: 0.8519
Epoch 2/20
4/4 [==============================] - 0s 13ms/step - loss: 0.7675 - accuracy: 0.8131 - val_loss: 0.6632 - val_accuracy: 0.8519
Epoch 3/20
4/4 [==============================] - 0s 14ms/step - loss: 0.7244 - accuracy: 0.8131 - val_loss: 0.6510 - val_accuracy: 0.8519
Epoch 4/20
4/4 [==============================] - 0s 16ms/step - loss: 0.7029 - accuracy: 0.8131 - val_loss: 0.6528 - val_accuracy: 0.8519
Epoch 5/20
4/4 [==============================] - 0s 14ms/step - loss: 0.6962 - accuracy: 0.8131 - val_loss: 0.6336 - val_accuracy: 0.8519
Epoch 6/20
4/4 [==============================] - 0s 14ms/step - loss: 0.6813 - a